# Introducción
En este Notebook aprenderemos las operaciones básicas más utilizadas en PySpark, incluyendo un ejercicio práctico en el que realizaremos una pequeña ETL (Extract Transform Load) para extraer unos datos desde una API y los volcaremos en el catálogo de datos de Spark.

No olvidemos que Apache Spark es una herramienta para el procesamiento distribuido de datos, es decir, está pensada para ser desplegada en un clúster (un conjunto de computadores) de modo que todas las operaciones de transformación de datos sean ejecutadas a lo largo de los diferentes nodos de este clúster de manera paralela. Sin embargo, nosotros no vamos a ejecutar este notebook sobre ningún clúster, ya que ello implicaría un cierto coste económico, lo que no es necesario para el pequeño volúmen de datos que manejaremos en este curso introductorio.

Por ello, a lo largo de este Notebook, la arquitectura de Spark únicamente consistirá de un driver, que se corresponderá con proceso del Sistema Operativo encargado de ejecutar el propio Notebook. No habrá ningún otro worker.

Cabe mencionar también que Apache Spark, aunque potencialmente puede correr bajo cualquier sistema operativo, se encuentra especialmente diseñado para entornos Linux, que es el sistema operativo más extendido entre servidores. Por ello se encomienda al alumno a ejecutar este Notebook en un sistema Linux (puede utilizarse WSL en el caso de disponer de Windows); tanto Google Colab como Kaggle utilizan Linux internamente, así que ambas plataformas son perfectamente válidas.

# Instalación de paquetes
En el caso de estar ejecutando este Notebook a través de una plataforma Cloud como Google Colab o Kaggle, será necesario instalar el paquete de Python desarrollado en este repositorio. Como se trata de un repositorio público, el gestor de paquetes de Python, `pip`, puede instalarlo directamente utilizando la herramienta de control de versiones más popular, `git`. 

Para ello, descomenta el siguiente código y ejecútalo (únicamente si tu entorno de ejecución es Cloud):

In [1]:
# !pip install git+https://github.com/donielix/esic-bigdata-iv-blackops.git > /dev/null

# Importación de módulos requeridos
En primer lugar necesitamos importar las funciones y objetos requeridos para el desarrollo del Notebook.

- `SparkSession`: objeto necesario para la interacción con la herramienta de Spark a través de Python. Es el punto de entrada principal.
- `pyspark.sql.functions`: funciones de SQL que ofrece pyspark, necesarias para las transformaciones de los datos en la ETL. Dado que esta colección de funciones SQL es utilizada recurrentemente, importamos el módulo completo con un alias que denotamos por `f`. Ello facilitará posteriormente el acceso a sus funciones utilizando el acceso a los atributos mediante un signo de puntuación, propio de Python.
- `fetch_api, save_json`: estas funciones están definidas dentro de nuestra propia librería llamada `blackops`, dentro de este mismo repositorio. Contienen el código necesario para extraer y almacenar los datos de la API. Notar cómo la ruta de importación sigue el arbol de carpetas del repositorio.
- `date, timedelta`: funciones para crear objetos de tipo fecha y timestamp dentro de Python.
- `random`: módulo utilizado para la generación de datos aleatorios. Así podremos crear una colección de datos de manera aleatoria, sin tener que introducir cada registro a mano.
- `DeltaTable`: objeto para interaccionar con tablas de tipo Delta. Se trata de un formato ampliamente utilizado en Spark, que ofrece muchas funcionalidades añadidas a nuestro catálogo de datos, como por ejemplo la posibilidad de revertir cambios.

In [2]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as f
from blackops.crawlers.wallapop.functions import fetch_api
from blackops.utils.io import save_json
from blackops.utils.catalog import get_detailed_tables_info
from datetime import date, timedelta
import random
from delta import DeltaTable

Establecemos una semilla para la generación de números aleatorios. De esta manera, los resultados serán reproducibles

In [3]:
random.seed(45)

# Inicialización de la sesión de Spark

Establecemos ahora la comunicación con el motor de Spark desde Python, a través del objeto `SparkSession` de la librería `pyspark`.

Como ya hemos comentado, en este caso no estamos utilizando un clúster, sino que haremos uso de una arquitectura local. El propio Jupyter Notebook ejercerá como Driver, como Master y como Ejecutor de las tareas, y ello se explicita en el método `.master("local[*]")`.

Adicionalmente, estamos instalando dependencias externas como la librería Delta, que incorpora utilidades muy importantes para el manejo de las tablas en nuestro catálogo de datos (histórico de versiones de tablas, omisión de ficheros innecesarios en la lectura, etc.)

In [4]:
spark = (
    SparkSession.Builder()
    .master("local[*]")
    .config(
        map={
            "spark.driver.memory": "8g",
            "spark.jars.packages": "io.delta:delta-spark_2.12:3.2.0",
            "spark.sql.extensions": "io.delta.sql.DeltaSparkSessionExtension",
            "spark.sql.catalog.spark_catalog": "org.apache.spark.sql.delta.catalog.DeltaCatalog",
            "spark.databricks.delta.retentionDurationCheck.enabled": "false",
            "spark.sql.catalogImplementation": "hive",
            "spark.sql.repl.eagerEval.enabled": "true",
            "spark.sql.repl.eagerEval.truncate": "100",
        }
    )
    .getOrCreate()
)

24/10/08 17:22:32 WARN Utils: Your hostname, pop-os resolves to a loopback address: 127.0.1.1; using 192.168.1.38 instead (on interface enp3s0)
24/10/08 17:22:32 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/dadiego/projects/ESIC/esic-bigdata-iv-blackops/.venv/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/dadiego/.ivy2/cache
The jars for the packages stored in: /home/dadiego/.ivy2/jars
io.delta#delta-spark_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-eb659fa6-3503-482a-a023-1476a6b511a8;1.0
	confs: [default]
	found io.delta#delta-spark_2.12;3.2.0 in central
	found io.delta#delta-storage;3.2.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
:: resolution report :: resolve 124ms :: artifacts dl 7ms
	:: modules in use:
	io.delta#delta-spark_2.12;3.2.0 from central in [default]
	io.delta#delta-storage;3.2.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   

Una vez se ha inicializado la sesión, podemos acceder a la web `localhost:4040` para consultar la interfaz de administración que ofrece Spark. Allí, se podrá monitorizar las tareas que se mandan desde el Driver.

**Nota**: Si al inicializar la sesión de Spark obtenemos algún error en el que se nos indica que la variable `JAVA_HOME` no existe, lo más probable es que no tengamos instalado Java en nuestro sistema, y necesitamos instalarlo ya que Spark depende de Java para su funcionamiento. Para ello, en Linux podemos utilizar el gestor de paquetes: `sudo apt update && sudo apt install openjdk-17-jdk -y`.

# Creación de un DataFrame de Spark

En Spark podemos crear directamente un Dataframe a partir de una lista de datos, o bien de un Dataframe de pandas. Para ello se puede utilizar el método `spark.createDataFrame`.
Debemos especificar tanto los datos como el esquema que tiene el Dataframe (sus columnas y sus tipos).

En este caso hacemos uso del paquete `random` para generar datos aleatorios (pero reproducibles, al haber establecido una semilla).

In [5]:
# Podemos especificar el esquema del DataFrame usando una cadena de texto
schema = "id INT, nombre STRING, edad INT, salario FLOAT, es_empleado BOOLEAN, fecha_contratacion DATE, departamento STRING"

# Crear una lista de datos ficticios
nombres = [
    "Juan",
    "María",
    "Pedro",
    "Ana",
    "Luis",
    "Carla",
    "Miguel",
    "Sara",
    "David",
    "Laura",
]
departamentos = ["Ventas", "Marketing", "Finanzas", "IT", "RRHH"]

data = [
    (
        i,  # id
        random.choice(nombres),  # nombre
        random.randint(22, 60),  # edad
        round(random.uniform(20000, 80000), 2),  # salario
        random.choice([True, False]),  # es_empleado
        date(2024, 10, 1)
        - timedelta(days=random.randint(0, 3650)),  # fecha_contratacion
        random.choice(departamentos),  # departamento
    )
    for i in range(1, 31)  # Genera 30 registros aleatorios
]

# Crear el DataFrame usando el esquema en string
df = spark.createDataFrame(data, schema)

# Creamos una vista temporal del DataFrame en el catálogo, para poder hacer consultas en SQL.
df.createOrReplaceTempView("empleados")

# Mostramos el DataFrame resultante por pantalla
display(df)

id,nombre,edad,salario,es_empleado,fecha_contratacion,departamento
1,Luis,48,49281.69,true,2021-05-09,Finanzas
2,Juan,26,49055.36,true,2021-07-27,Ventas
3,Luis,24,73947.68,true,2023-03-28,Finanzas
4,Pedro,35,79554.92,false,2023-11-29,RRHH
5,Miguel,31,62027.07,true,2022-10-27,Finanzas
6,Luis,44,66505.58,false,2023-09-13,IT
7,María,23,65070.76,false,2015-12-18,IT
8,David,56,72059.12,false,2018-02-06,Ventas
9,Sara,45,74705.86,false,2023-01-02,Ventas
10,Ana,53,62691.89,false,2021-06-16,Marketing


Recordemos que Spark procesa los datos de manera distribuida, en lo que se conoce como particiones. Por eso, aunque nosotros visualicemos la tabla anterior en su conjunto, cuando la ejecución sucede en un clúster, sucederá que unos registros de este DataFrame estarán almacenados en un computador y otros residirán en otro distinto. Podemos comprobar cuántas particiones existen en nuestra tabla con el siguiente comando

In [6]:
df.rdd.getNumPartitions()

16

Incluso podemos ver a qué partición corresponde cada registro

In [10]:
df.withColumn("id_particion", f.spark_partition_id())

id,nombre,edad,salario,es_empleado,fecha_contratacion,departamento,id_particion
1,Luis,48,49281.69,true,2021-05-09,Finanzas,0
2,Juan,26,49055.36,true,2021-07-27,Ventas,1
3,Luis,24,73947.68,true,2023-03-28,Finanzas,1
4,Pedro,35,79554.92,false,2023-11-29,RRHH,2
5,Miguel,31,62027.07,true,2022-10-27,Finanzas,2
6,Luis,44,66505.58,false,2023-09-13,IT,3
7,María,23,65070.76,false,2015-12-18,IT,3
8,David,56,72059.12,false,2018-02-06,Ventas,4
9,Sara,45,74705.86,false,2023-01-02,Ventas,4
10,Ana,53,62691.89,false,2021-06-16,Marketing,5


# Operaciones de transformación

La sintaxis de Spark es muy similar a la del lenguaje SQL, de hecho, admite la introducción de comandos SQL para realizar las transformaciones de los datos. Vamos a ver algunas de las operaciones más habituales.

### Select

La operación más sencilla consiste en seleccionar simplemente un subconjunto de los datos, sin ninguna otra operación de transformación o filtro añadido. Por ejemplo, seleccionemos únicamente los campos `id` y `nombre`.

In [5]:
df.select("id", "nombre")

id,nombre
1,Luis
2,Juan
3,Luis
4,Pedro
5,Miguel
6,Luis
7,María
8,David
9,Sara
10,Ana


Al igual que en SQL estándar, podemos no solo seleccionar unas columnas sino aplicarles alguna función de transformación dentro del propio comando SELECT, y renombrarlas utilizando un alias.

Las funciones SQL en Spark están contenidas en el módulo `pyspark.sql.functions`, que hemos importado al principio y lo hemos almacenado en un objeto con alias `f` (por sencillez de uso).

Vamos a seleccionar en este caso los mismos campos que en el ejemplo anterior, sin embargo, al campo `nombre` le vamos a aplicar una transformación para visualizar el nombre en mayúsculas, y al resultado lo renombraremos `nombre_en_mayusculas`.

In [6]:
df.select("id", f.upper("nombre").alias("nombre_en_mayusculas"))

id,nombre_en_mayusculas
1,LUIS
2,JUAN
3,LUIS
4,PEDRO
5,MIGUEL
6,LUIS
7,MARÍA
8,DAVID
9,SARA
10,ANA


### WithColumn
Podemos añadir campos nuevos derivados a partir de otros campos utilizando el método `withColumn`. Este comando conservará todas las columnas de la tabla, y añadirá una adicional, con las transformaciones que le indiquemos.

Por ejemplo, en nuestra tabla disponemos del campo `edad`, pero supongamos que nos interesa, para nuestra analítica, disponer de un campo con el año de nacimiento. En tal caso, podríamos concatenar dos funciones SQL: con la primera, `current_date`, extraemos la fecha actual, y sobre dicha fecha aplicamos la función `year` para extraer el año. Finalmente, a este año actual le restamos la edad que tiene el usuario para así calcular su año de nacimiento. Cada usuario dispondrá así de un año de nacimiento (transformación fila a fila).

In [7]:
df.withColumn("año_nacimiento", f.year(f.current_date()) - f.col("edad"))

id,nombre,edad,salario,es_empleado,fecha_contratacion,departamento,año_nacimiento
1,Luis,48,49281.69,true,2021-05-09,Finanzas,1976
2,Juan,26,49055.36,true,2021-07-27,Ventas,1998
3,Luis,24,73947.68,true,2023-03-28,Finanzas,2000
4,Pedro,35,79554.92,false,2023-11-29,RRHH,1989
5,Miguel,31,62027.07,true,2022-10-27,Finanzas,1993
6,Luis,44,66505.58,false,2023-09-13,IT,1980
7,María,23,65070.76,false,2015-12-18,IT,2001
8,David,56,72059.12,false,2018-02-06,Ventas,1968
9,Sara,45,74705.86,false,2023-01-02,Ventas,1979
10,Ana,53,62691.89,false,2021-06-16,Marketing,1971


### Filter

Podemos filtrar los datos de acuerdo a alguna condición especificada. Esta sentencia se corresponde con el comando `WHERE` en SQL. Por ejemplo, queremos obtener únicamente los datos de los empleados. 

Recordemos que en Python el operador de igualdad es `==`.

Para poder realizar operaciones con columnas, necesitamos especificar que se trata de una columna del DataFrame haciendo uso de la función `col`, puesto que si no lo que estaríamos es comparando un string con un booleano (`"es_empleado" == True`), que será siempre igual a `False`.

In [8]:
df.filter(f.col("es_empleado") == True)

# Si hacemos df.filter("es_empleado" == True) obtendremos un error porque los tipos no son los esperados.

id,nombre,edad,salario,es_empleado,fecha_contratacion,departamento
1,Luis,48,49281.69,true,2021-05-09,Finanzas
2,Juan,26,49055.36,true,2021-07-27,Ventas
3,Luis,24,73947.68,true,2023-03-28,Finanzas
5,Miguel,31,62027.07,true,2022-10-27,Finanzas
11,Miguel,46,70126.54,true,2018-03-14,IT
12,David,27,53608.56,true,2014-11-09,RRHH
17,Ana,53,21659.3,true,2018-01-19,Finanzas
18,Sara,28,30491.18,true,2015-12-13,Ventas
23,Carla,50,73172.89,true,2015-08-21,IT
25,Sara,59,59973.55,true,2018-04-23,Marketing


### Agrupaciones

Utilizando el comando group by, podemos agrupar nuestro dataset según los valores de una o varias columnas y posteriormente realizar una operación de agregación sobre cada conjunto, para así obtener estadísticas descriptivas de nuestros datos.

Por ejemplo, podemos obtener el número de empleados en marketing, con lo cual debemos agrupar por departamento y realizar una operación de agregación de suma. Estas operaciones se denominan "de agregación" o "de reducción" porque actúan sobre un conjunto de filas (todas aquellas que comparten el mismo valor del grupo) y devuelven un único valor

In [9]:
df.groupBy("departamento").agg(f.sum("salario").alias("salario_total"))

departamento,salario_total
Finanzas,524206.53515625
Ventas,421618.7890625
RRHH,292734.91015625
IT,355377.107421875
Marketing,122665.44140625


### Combinaciones
Naturalmente, la riqueza de PySpark es que podemos combinar filtros con agrupaciones, adición de columnas, cambios de tipos, etc para que nuestro dato final quede pulido.

Al contrario que en Pandas, todas las operaciones de transformación en Spark son *lazy*, es decir, no se evalúan hasta que se pide una acción (resultado). Esto permite que el catalizador de Spark optimice toda la cadena de consultas de la manera más apropiada antes de ser ejecutadas.

Veamos un ejemplo de consulta algo más avanzada: supongamos que queremos conocer cuál es el departamento del que más gente se ha ido a partir de 2017 para unos ciertos intervalos de meses: enero a mayo, junio a septiembre y octubre a diciembre. En este caso podemos comenzar aplicando unos filtros para quedarnos únicamente con registros de los que actualmente ya no son empleados y su fecha de contratación es igual o posterior a 2017. Después de aplicar dicho filtro, podemos añadir dos columnas transitorias para extraer el mes de la fecha de contratación y establecer los intervalos pedidos, utilizando la función `when`, que es esquivalente al `CASE` de SQL. Finalmente, agrupamos por estas categorías de mes y agregamos cogiendo la moda (el valor más repetido de un conjunto de datos).

In [10]:
df.filter(
    (f.col("es_empleado") == False) & (f.year("fecha_contratacion") >= 2017)
).withColumn("mes_contratacion", f.month("fecha_contratacion")).withColumn(
    "categoria_mes",
    f.when(f.col("mes_contratacion").between(1, 5), f.lit("enero-mayo"))
    .when(f.col("mes_contratacion").between(6, 9), f.lit("junio-septiembre"))
    .otherwise(f.lit("octubre-diciembre")),
).groupBy(
    "categoria_mes"
).agg(
    f.mode("departamento").alias("departamento_mas_repetido")
)

categoria_mes,departamento_mas_repetido
octubre-diciembre,RRHH
junio-septiembre,Marketing
enero-mayo,Ventas


La consulta equivalente en Spark SQL en este caso sería la siguiente

In [11]:
spark.sql(
    """
    SELECT
        CASE
            WHEN MONTH(fecha_contratacion) BETWEEN 1 AND 5 THEN 'enero-mayo'
            WHEN MONTH(fecha_contratacion) BETWEEN 6 AND 9 THEN 'junio-septiembre'
            ELSE 'octubre-diciembre'
        END AS categoria_mes,
        MODE(departamento) AS departamento_mas_repetido

    FROM empleados
    WHERE
        es_empleado = false AND
        YEAR(fecha_contratacion) >= 2017
    GROUP BY categoria_mes
    """
)

categoria_mes,departamento_mas_repetido
octubre-diciembre,RRHH
junio-septiembre,Marketing
enero-mayo,Ventas


Como se puede comprobar, se obtienen exactamente los mismos resultados.

Disgreguemos la consulta compleja de pyspark en componentes, para ver los resultados de aplicar cada una de las transformaciones. Empezamos con el filtro para quedarnos con registros de personas que ya no son empleadas y que fueron contratadas a partir del año 2017

In [12]:
df.filter(
    (f.col("es_empleado") == False) & (f.year("fecha_contratacion") >= 2017)
)

id,nombre,edad,salario,es_empleado,fecha_contratacion,departamento
4,Pedro,35,79554.92,false,2023-11-29,RRHH
6,Luis,44,66505.58,false,2023-09-13,IT
8,David,56,72059.12,false,2018-02-06,Ventas
9,Sara,45,74705.86,false,2023-01-02,Ventas
10,Ana,53,62691.89,false,2021-06-16,Marketing
13,Laura,23,57455.18,false,2021-03-08,Finanzas
14,Pedro,33,20602.28,false,2024-03-12,IT
15,María,22,57444.64,false,2017-10-22,Finanzas
16,Sara,50,32029.18,false,2022-01-13,RRHH
21,Luis,46,65993.36,false,2022-12-04,RRHH


Posteriormente, con el comando `withColumn`, estamos añadiendo un nuevo campo que extrae el mes a partir de la fecha de contratación

In [13]:
df.filter(
    (f.col("es_empleado") == False) & (f.year("fecha_contratacion") >= 2017)
).withColumn("mes_contratacion", f.month("fecha_contratacion"))

id,nombre,edad,salario,es_empleado,fecha_contratacion,departamento,mes_contratacion
4,Pedro,35,79554.92,false,2023-11-29,RRHH,11
6,Luis,44,66505.58,false,2023-09-13,IT,9
8,David,56,72059.12,false,2018-02-06,Ventas,2
9,Sara,45,74705.86,false,2023-01-02,Ventas,1
10,Ana,53,62691.89,false,2021-06-16,Marketing,6
13,Laura,23,57455.18,false,2021-03-08,Finanzas,3
14,Pedro,33,20602.28,false,2024-03-12,IT,3
15,María,22,57444.64,false,2017-10-22,Finanzas,10
16,Sara,50,32029.18,false,2022-01-13,RRHH,1
21,Luis,46,65993.36,false,2022-12-04,RRHH,12


Tras esto, de nuevo con el método `withColumn`, añadimos otra columna que establecerá la categoría del mes, en función del campo `mes_contratacion` anteriormente calculado, y el resultado será el siguiente

In [14]:
df.filter(
    (f.col("es_empleado") == False) & (f.year("fecha_contratacion") >= 2017)
).withColumn("mes_contratacion", f.month("fecha_contratacion")).withColumn(
    "categoria_mes",
    f.when(f.col("mes_contratacion").between(1, 5), f.lit("enero-mayo"))
    .when(f.col("mes_contratacion").between(6, 9), f.lit("junio-septiembre"))
    .otherwise(f.lit("octubre-diciembre")),
)

id,nombre,edad,salario,es_empleado,fecha_contratacion,departamento,mes_contratacion,categoria_mes
4,Pedro,35,79554.92,false,2023-11-29,RRHH,11,octubre-diciembre
6,Luis,44,66505.58,false,2023-09-13,IT,9,junio-septiembre
8,David,56,72059.12,false,2018-02-06,Ventas,2,enero-mayo
9,Sara,45,74705.86,false,2023-01-02,Ventas,1,enero-mayo
10,Ana,53,62691.89,false,2021-06-16,Marketing,6,junio-septiembre
13,Laura,23,57455.18,false,2021-03-08,Finanzas,3,enero-mayo
14,Pedro,33,20602.28,false,2024-03-12,IT,3,enero-mayo
15,María,22,57444.64,false,2017-10-22,Finanzas,10,octubre-diciembre
16,Sara,50,32029.18,false,2022-01-13,RRHH,1,enero-mayo
21,Luis,46,65993.36,false,2022-12-04,RRHH,12,octubre-diciembre


Por último, sobre este último DataFrame, se hace una agrupación según la categoría del mes recientemente calculada, y se computa la moda del campo `departamento` para cada agrupación, obteniendo así la tabla final que hemos visto arriba

# Joins
Al igual que las operaciones de transformación, otro comando importante es el de JOIN, que nos permite establecer links entre campos de diferentes columnas, lo cual resulta fundamental para el análisis desde diferentes fuentes de datos.

Un análisis detallado y extenso de los diferentes tipos de Joins en Pyspark puede verse [aquí](https://sparkbyexamples.com/pyspark/pyspark-join-explained-with-examples/).

Esquemáticamente tenemos los siguientes tipos de JOINs:

![tipos de joins](joins.png "Tipos de Joins")

Por poner un ejemplo, creemos un segundo dataframe con información añadida de los departamentos

In [15]:
departamentos = spark.createDataFrame(
    [
        (
            1,
            "Finanzas",
            "Departamento encargado de elaborar informes financieros trimestrales",
        ),
        (
            2,
            "Ventas",
            "Departamento encargado de contactar con proveedores y registrar el stock",
        ),
    ], schema="id int, departamento string, descripcion string"
)
display(departamentos)

id,departamento,descripcion
1,Finanzas,Departamento encargado de elaborar informes financieros trimestrales
2,Ventas,Departamento encargado de contactar con proveedores y registrar el stock


Y ahora vamos a unir nuestra tabla de origen con esta tabla de información extendida por departamento. El campo de unión lógicamente será la columna `"departamento"`.

Empecemos con un INNER JOIN. En este caso, únicamente se mostrarán los departamentos de Finanzas y Ventas, ya que son los únicos registros comunes a ambas tablas (el resto de departamentos no está presente en la tabla `departamentos`)

In [16]:
df.join(departamentos, how="inner", on="departamento")

departamento,id,nombre,edad,salario,es_empleado,fecha_contratacion,id,descripcion
Finanzas,1,Luis,48,49281.69,true,2021-05-09,1,Departamento encargado de elaborar informes financieros trimestrales
Finanzas,3,Luis,24,73947.68,true,2023-03-28,1,Departamento encargado de elaborar informes financieros trimestrales
Finanzas,5,Miguel,31,62027.07,true,2022-10-27,1,Departamento encargado de elaborar informes financieros trimestrales
Finanzas,13,Laura,23,57455.18,false,2021-03-08,1,Departamento encargado de elaborar informes financieros trimestrales
Finanzas,15,María,22,57444.64,false,2017-10-22,1,Departamento encargado de elaborar informes financieros trimestrales
Finanzas,17,Ana,53,21659.3,true,2018-01-19,1,Departamento encargado de elaborar informes financieros trimestrales
Finanzas,22,Juan,26,39267.68,false,2022-02-16,1,Departamento encargado de elaborar informes financieros trimestrales
Finanzas,26,Ana,25,22953.8,false,2018-05-11,1,Departamento encargado de elaborar informes financieros trimestrales
Finanzas,29,Miguel,37,69922.06,true,2023-10-26,1,Departamento encargado de elaborar informes financieros trimestrales
Finanzas,30,Luis,27,70247.43,true,2020-05-23,1,Departamento encargado de elaborar informes financieros trimestrales


Ahora haremos un LEFT JOIN. En este caso se mostrarán todos los registros de la tabla de origen, con todos los departamentos por tanto. Y si existe un departamento equivalente en la tabla derecha, se mostrará también su información adicional. En caso de no existir un match (cuando el departamento no sea el de ventas o finanzas), dicha información adicional será nula

In [17]:
df.join(departamentos, how="left", on="departamento")

departamento,id,nombre,edad,salario,es_empleado,fecha_contratacion,id,descripcion
Finanzas,1,Luis,48,49281.69,true,2021-05-09,1,Departamento encargado de elaborar informes financieros trimestrales
Finanzas,3,Luis,24,73947.68,true,2023-03-28,1,Departamento encargado de elaborar informes financieros trimestrales
Ventas,2,Juan,26,49055.36,true,2021-07-27,2,Departamento encargado de contactar con proveedores y registrar el stock
Finanzas,5,Miguel,31,62027.07,true,2022-10-27,1,Departamento encargado de elaborar informes financieros trimestrales
RRHH,4,Pedro,35,79554.92,false,2023-11-29,NULL,NULL
IT,6,Luis,44,66505.58,false,2023-09-13,NULL,NULL
IT,7,María,23,65070.76,false,2015-12-18,NULL,NULL
Ventas,8,David,56,72059.12,false,2018-02-06,2,Departamento encargado de contactar con proveedores y registrar el stock
Ventas,9,Sara,45,74705.86,false,2023-01-02,2,Departamento encargado de contactar con proveedores y registrar el stock
Marketing,10,Ana,53,62691.89,false,2021-06-16,NULL,NULL


Por último, veamos el caso del LEFT ANTI JOIN. En este caso, se mostrarán únicamente los registros de la tabla origen que no tienen un match con los de la tabla de departamentos; es decir, aquellos registros del dataframe cuyo departamento no es ventas ni finanzas

In [18]:
df.join(departamentos, how="leftanti", on="departamento")

departamento,id,nombre,edad,salario,es_empleado,fecha_contratacion
RRHH,4,Pedro,35,79554.92,false,2023-11-29
IT,6,Luis,44,66505.58,false,2023-09-13
IT,7,María,23,65070.76,false,2015-12-18
Marketing,10,Ana,53,62691.89,false,2021-06-16
IT,11,Miguel,46,70126.54,true,2018-03-14
RRHH,12,David,27,53608.56,true,2014-11-09
IT,14,Pedro,33,20602.28,false,2024-03-12
RRHH,16,Sara,50,32029.18,false,2022-01-13
IT,20,Pedro,27,59899.06,false,2015-02-25
RRHH,19,María,26,61548.89,false,2015-11-03


# Caso práctico: Extracción de datos de Wallapop
Vamos a construir un pequeño ejemplo de una ETL (Extraction Transform Load). Extraeremos datos en crudo desde la API REST de Wallapop, los guardamos en una carpeta de almacenamiento, los leemos con spark, realizamos algunas transformaciones y almacenamos la tabla resultante en nuestro catálogo de datos

In [19]:
try:
    json_data = fetch_api(product="portátil")
    save_json(obj=json_data, path="data/wallapop.json", indent=4)
except Exception as e:
    print(f"Warning: No ha sido posible descargar los datos de la API: {e}")

Podemos previsualizar cuál es la estructura de nuestro fichero JSON utilizando el comando externo `cat` de nuestra terminal (válido únicamente en sistemas Unix, con `jq` instalado).

Si no está instalado `jq`, puede instalarse mediante `sudo apt update && sudo apt install jq -y`

In [20]:
%%sh
cat data/wallapop.json | jq -C | head -20

{
  "data": {
    "section": {
      "payload": {
        "order": "most_relevance",
        "title": "Find what you want",
        "items": [
          {
            "id": "8j34odd1ey69",
            "user_id": "kp61on7oqxj5",
            "title": "IVEOPPE teclado portatil",
            "description": "IVEOPPE teclado portatil samsung galaxy TAB a9 + carcasa\na estrenar\nprecio en amazon: 30,99€",
            "category_id": 24200,
            "price": {
              "amount": 15,
              "currency": "EUR"
            },
            "images": [
              {
                "average_color": "13C1AC",


Una vez determinada la estructura que tiene nuestro fichero JSON de información, notamos que los datos que queremos obtener se encuentran dentro de la ruta `data -> section -> payload -> items`. Dicha ruta se corresponde con un array (lista) de items, que son los productos de Wallapop; cada uno de ellos tiene unos campos, algunos simples como `id`, `user_id`, y otros compuestos como `price -> amount` o `price -> currency`.

En primer lugar, observemos que si leemos el fichero JSON directamente no obtenemos una estructura muy amigable

*Nota*: el argumento `multiLine=True` se introduce para especificar que el fichero JSON de entrada que estamos tratando de leer abarca múltiples líneas, y no una sola. De esta manera nos evitaremos errores de lectura.

In [21]:
wallapop = spark.read.json("data/wallapop.json", multiLine=True)
wallapop.show(truncate=50)

+--------------------------------------------------+--------------------------------------------------+
|                                              data|                                              meta|
+--------------------------------------------------+--------------------------------------------------+
|{{{[{{none}, 24200, 1728317203080, IVEOPPE tecl...|{eyJhbGciOiJIUzI1NiJ9.eyJwYXJhbXMiOnsic2VhcmNoU...|
+--------------------------------------------------+--------------------------------------------------+



Esto es porque nos ha cogido las dos primeras claves más externas de nuestro fichero JSON, que son los campos `"data"` y `"meta"`.

Observemos qué estructura hemos cargado haciendo un `printSchema` de nuestro DataFrame. De esta manera obtendremos información de los campos y sus tipos

In [22]:
wallapop.printSchema()

root
 |-- data: struct (nullable = true)
 |    |-- section: struct (nullable = true)
 |    |    |-- payload: struct (nullable = true)
 |    |    |    |-- items: array (nullable = true)
 |    |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |    |-- bump: struct (nullable = true)
 |    |    |    |    |    |    |-- type: string (nullable = true)
 |    |    |    |    |    |-- category_id: long (nullable = true)
 |    |    |    |    |    |-- created_at: long (nullable = true)
 |    |    |    |    |    |-- description: string (nullable = true)
 |    |    |    |    |    |-- discount: struct (nullable = true)
 |    |    |    |    |    |    |-- percentage: long (nullable = true)
 |    |    |    |    |    |    |-- previous_price: struct (nullable = true)
 |    |    |    |    |    |    |    |-- amount: double (nullable = true)
 |    |    |    |    |    |    |    |-- currency: string (nullable = true)
 |    |    |    |    |    |-- favorited: struct (nullable = true)
 

Ahora, para navegar a través de nuestro fichero JSON, podemos utilizar la sintáxis por puntos; es decir, para obtener el campo deseado `"items"`, que contiene la información de todos los productos, debemos acceder mediante `data.section.payload.items`.

In [23]:
wallapop.select("data.section.payload.items").show(truncate=100)

+----------------------------------------------------------------------------------------------------+
|                                                                                               items|
+----------------------------------------------------------------------------------------------------+
|[{{none}, 24200, 1728317203080, IVEOPPE teclado portatil samsung galaxy TAB a9 + carcasa\na estre...|
+----------------------------------------------------------------------------------------------------+



Sin embargo, seguimos sin apreciar una estructura legible. Esto es porque se nos está mostrando un único registro (fila) que contiene toda la información de los productos. Lo que nos interesa es que cada elemento de esta lista se muestre en un registro a parte. Para ello se utiliza la función SQL `explode`, que coge un array de elementos y devuelve un registro por cada uno de esos elementos. Veámoslo

In [24]:
wallapop.select(f.explode("data.section.payload.items")).show(truncate=100)

+----------------------------------------------------------------------------------------------------+
|                                                                                                 col|
+----------------------------------------------------------------------------------------------------+
|{{none}, 24200, 1728317203080, IVEOPPE teclado portatil samsung galaxy TAB a9 + carcasa\na estren...|
|{{none}, 24200, 1728316393445, Mcbazel \nTeclado portatil plegable recargable con dispositivo tac...|
|{{none}, 24200, 1728327614065, Se vende porque se le da uso está en perfectas condiciones poco us...|
|{{none}, 24200, 1728325351863, Se vende porque no se le da uso está en perfectas condiciones poco...|
|{{none}, 24200, 1728304636450, Monitor nuevo. Contiene, pantalla, respaldo de pantalla, cable tip...|
|{{none}, 24200, 1728327810358, Funciona, aunque va muy lento. los botones del tactil van mal. Se ...|
|{{none}, 24200, 1728325826012, Portatil Acer funciona con su cargador or

Bien, ya hemos avanzado, disponemos ahora de un registro por cada producto de la lista `items`, como queríamos. Sin embargo, se sigue mostrando toda la información en una misma columna. Eso lo solucionamos seleccionando los campos anidados deseados. Por ejemplo, supongamos que queremos coger el `id` del producto, el `user_id` del usuario y la fecha de creación del anuncio `created_at`. Una buena manera de operar sería crear una nueva columna llamada, por ejemplo, `"data"`, que contenga los registros explotados del campo de `"items"`, y luego utilizar este nuevo campo para obtener la info de los otros campos descendientes

In [25]:
wallapop.withColumn("data", f.explode("data.section.payload.items")).select(
    "data.id", "data.user_id", "data.created_at"
)

id,user_id,created_at
8j34odd1ey69,kp61on7oqxj5,1728317203080
e6582kk9ok6o,kp61on7oqxj5,1728316393445
3zl8xwq1np6x,xzo8x8v9gl69,1728327614065
xzo2omn38w69,nz0m0n7l93jo,1728325351863
nzx4vn9247j2,8z812r1reo63,1728304636450
8z8k9nm8v1z3,qzm4m7kvwgzv,1728327810358
xzo2omnoo469,3zlgnx7x3njx,1728325826012
w6749l0nyy6x,8ejkpqn079zx,1728149565059
nzx4vnqe55j2,8z812r2my163,1728298268913
3zl8xw3y1p6x,e65y9rpllgjo,1728321679391


Fenomenal. Ahora, siguiendo esta misma operación, obtendremos un dataset completo tabular de los campos del JSON más relevantes. Observemos que para todos los campos seleccionados, se hace una conversión de tipos (método `cast`) y se asigna un alias (método `alias`). Esto es para que la tabla resultante sea consistente, y tenga siempre el mismo esquema de salida.

Notemos también que a campos que representan fechas pero se muestran como números enteros (milisegundos desde 1970, esto se conoce como UNIX time), como `created_at` o `modified_at`, les aplicamos una conversión mediante la función `from_unixtime` para representarlos como una fecha legible

In [26]:
wallapop = (
    spark.read.json("data/wallapop.json", multiLine=True, primitivesAsString=True)
    .withColumn("data", f.explode("data.section.payload.items"))
    .select(
        f.col("data.id").cast("string").alias("id"),
        f.col("data.title").cast("string").alias("title"),
        f.col("data.user_id").cast("string").alias("user_id"),
        f.col("data.category_id").cast("int").alias("category_id"),
        f.from_unixtime((f.col("data.created_at") / 1000))
        .cast("timestamp")
        .alias("created_at"),
        f.from_unixtime(f.col("data.modified_at") / 1000)
        .cast("timestamp")
        .alias("modified_at"),
        f.col("data.description").cast("string").alias("description"),
        f.col("data.favorited.flag").cast("boolean").alias("favorited"),
        f.col("data.is_favoriteable.flag").cast("boolean").alias("is_favoriteable"),
        f.col("data.is_refurbished.flag").cast("boolean").alias("is_refurbished"),
        f.col("data.location.latitude").cast("double").alias("latitude"),
        f.col("data.location.longitude").cast("double").alias("longitude"),
        f.col("data.location.postal_code").cast("string").alias("postal_code"),
        f.col("data.location.city").cast("string").alias("city"),
        f.col("data.location.region").cast("string").alias("region"),
        f.col("data.location.region2").cast("string").alias("region2"),
        f.col("data.location.country_code").cast("string").alias("country_code"),
        f.col("data.price.amount").cast("double").alias("amount"),
        f.col("data.price.currency").cast("string").alias("currency"),
        f.col("data.reserved.flag").alias("reserved"),
        f.col("data.shipping.item_is_shippable")
        .cast("boolean")
        .alias("item_is_shippable"),
        f.col("data.shipping.user_allows_shipping")
        .cast("boolean")
        .alias("user_allows_shipping"),
        f.current_timestamp().alias("__timestamp"),
    )
)
wallapop.printSchema()
display(wallapop)

root
 |-- id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- category_id: integer (nullable = true)
 |-- created_at: timestamp (nullable = true)
 |-- modified_at: timestamp (nullable = true)
 |-- description: string (nullable = true)
 |-- favorited: boolean (nullable = true)
 |-- is_favoriteable: boolean (nullable = true)
 |-- is_refurbished: boolean (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- postal_code: string (nullable = true)
 |-- city: string (nullable = true)
 |-- region: string (nullable = true)
 |-- region2: string (nullable = true)
 |-- country_code: string (nullable = true)
 |-- amount: double (nullable = true)
 |-- currency: string (nullable = true)
 |-- reserved: string (nullable = true)
 |-- item_is_shippable: boolean (nullable = true)
 |-- user_allows_shipping: boolean (nullable = true)
 |-- __timestamp: timestamp (nullable = false)



id,title,user_id,category_id,created_at,modified_at,description,favorited,is_favoriteable,is_refurbished,latitude,longitude,postal_code,city,region,region2,country_code,amount,currency,reserved,item_is_shippable,user_allows_shipping,__timestamp
8j34odd1ey69,IVEOPPE teclado portatil,kp61on7oqxj5,24200,2024-10-07 18:06:43,2024-10-07 18:06:53,"IVEOPPE teclado portatil samsung galaxy TAB a9 + carcasa\na estrenar\nprecio en amazon: 30,99€",false,true,false,40.41883190365425,-3.57184367222341,28821,Coslada,Comunidad de Madrid,Madrid,ES,15.0,EUR,false,true,false,2024-10-07 21:07:42.585303
e6582kk9ok6o,Teclado portatil plegable recargable,kp61on7oqxj5,24200,2024-10-07 17:53:13,2024-10-07 17:53:19,Mcbazel \nTeclado portatil plegable recargable con dispositivo tactil para dispositivo movil/tabl...,false,true,false,40.41883190365425,-3.57184367222341,28821,Coslada,Comunidad de Madrid,Madrid,ES,15.0,EUR,false,true,false,2024-10-07 21:07:42.585303
3zl8xwq1np6x,Portatil,xzo8x8v9gl69,24200,2024-10-07 21:00:14,2024-10-07 21:00:24,Se vende porque se le da uso está en perfectas condiciones poco uso y cualquier cosa por chat pre...,false,true,false,40.4013020602374,-3.61266807783252,28032,Madrid,Comunidad de Madrid,Madrid,ES,110.0,EUR,false,true,false,2024-10-07 21:07:42.585303
xzo2omn38w69,Portátil,nz0m0n7l93jo,24200,2024-10-07 20:22:31,2024-10-07 20:27:58,Se vende porque no se le da uso está en perfectas condiciones poco uso y cualquier cosa por chat ...,false,true,false,40.40293549760407,-3.6036383752125976,28032,Madrid,Comunidad de Madrid,Madrid,ES,110.0,EUR,false,true,false,2024-10-07 21:07:42.585303
nzx4vn9247j2,Monitor portátil 11'6 pulgadas,8z812r1reo63,24200,2024-10-07 14:37:16,2024-10-07 14:39:53,"Monitor nuevo. Contiene, pantalla, respaldo de pantalla, cable tipo c, cable tipo c a usb, cable ...",false,true,false,40.34549239334717,-3.8199855046355746,28922,Alcorcón,Comunidad de Madrid,Madrid,ES,65.0,EUR,false,true,false,2024-10-07 21:07:42.585303
8z8k9nm8v1z3,Ordenador portatil,qzm4m7kvwgzv,24200,2024-10-07 21:03:30,2024-10-07 21:04:38,"Funciona, aunque va muy lento. los botones del tactil van mal. Se escuchan ofertas. Tengo el carg...",false,true,false,40.343563207976395,-3.709246913711613,28021,Madrid,Comunidad de Madrid,Madrid,ES,120.0,EUR,false,true,true,2024-10-07 21:07:42.585303
xzo2omnoo469,Portatil Acer,3zlgnx7x3njx,24200,2024-10-07 20:30:26,2024-10-07 20:55:10,Portatil Acer funciona con su cargador original\nescucho ofertas,false,true,false,40.326254,-3.768059,28911,Leganés,Comunidad de Madrid,Madrid,ES,30.0,EUR,false,true,false,2024-10-07 21:07:42.585303
w6749l0nyy6x,Portatil gaming,8ejkpqn079zx,24200,2024-10-05 19:32:45,2024-10-05 21:37:29,"Se vende portátil MSI GF75 Thin 10SDR, Intel Core i7, 1TB, NVIDIA GeForce GTX1660Ti/6GB, Windows ...",false,true,false,40.322746246594974,-3.854662525858509,28937,Móstoles,Comunidad de Madrid,Madrid,ES,825.0,EUR,false,true,true,2024-10-07 21:07:42.585303
nzx4vnqe55j2,"CUIUIC Monitor portátil de 15,6 Pulgadas",8z812r2my163,24200,2024-10-07 12:51:08,2024-10-07 12:51:19,"precintado\nsolo envio\n\nCUIUIC Monitor portátil de 15,6 Pulgadas con Pantalla Full HD USB-C, 19...",false,true,false,40.416156498413585,-3.7101967123015727,28070,Madrid,Comunidad de Madrid,Madrid,ES,55.0,EUR,false,true,true,2024-10-07 21:07:42.585303
3zl8xw3y1p6x,ORDENADOR PORTÁTIL,e65y9rpllgjo,24200,2024-10-07 19:21:19,2024-10-07 19:21:29,ORDENADOR PORTÁTIL CHUWI GEMIBOOK XPRO. DESCRIPCIÓN EN FOTOS. NUEVO.\nSOLO ENVIO.,false,true,false,40.457704764564234,-3.754137266350963,28040,Madrid,Comunidad de Madrid,Madrid,ES,190.0,EUR,false,true,true,2024-10-07 21:07:42.585303


Genial, ya hemos leído y transformado nuestro set de datos inicialmente desestructurado en una tabla bien estructurada, con unos campos y tipos fijados.

Ahora, para completar la ETL, almacenaremos esta tabla en el catálogo de datos de Spark. En este caso, al estar trabajando de manera local, dicho catálogo de datos estará localizado en esta misma ruta (`metastore_db/`, `spark-warehouse/`, `derby.log`), sin embargo, cuando trabajemos en un entorno corporativo, habitualmente el catálogo de datos se alojará en una arquitectura cloud, como AWS, Azure, GCP, Databricks, etc.

Aunque no es necesario, es una buena práctica crear en primera instancia la tabla Delta sobre la que escribiremos nuestro dataset, con un schema concreto, metadatos, etc.

In [27]:
dt = (
    DeltaTable.createIfNotExists(spark)
    .addColumns(wallapop.schema)
    .tableName("wallapop")
    .comment("Tabla de productos de Wallapop")
    .execute()
)

24/10/07 21:07:43 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
24/10/07 21:07:43 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist
24/10/07 21:07:44 WARN ObjectStore: Version information not found in metastore. hive.metastore.schema.verification is not enabled so recording the schema version 2.3.0
24/10/07 21:07:44 WARN ObjectStore: setMetaStoreSchemaVersion called but recording version is disabled: version = 2.3.0, comment = Set by MetaStore dadiego@127.0.1.1
24/10/07 21:07:44 WARN ObjectStore: Failed to get database default, returning NoSuchObjectException
24/10/07 21:07:46 WARN HiveExternalCatalog: Couldn't find corresponding Hive SerDe for data source provider delta. Persisting data source table `spark_catalog`.`default`.`wallapop` into Hive metastore in Spark SQL specific format, which is NOT compatible with Hive.
24/10/07 21:07:46 WARN SessionState: METASTORE_FILTER_HOOK will be ignored, since hive.security.authorization.manager is s

Con el método `createIfNotExists` estamos indicando que cree únicamente la tabla en el catálogo de datos si esta no existía previamente.

Una vez creada la tabla, insertaremos los datos de nuestro DataFrame mediante una operación `merge`. Para ello, identificamos en primer lugar cuáles son los campos que consituyen una clave primaria en la tabla, es decir, un identificador único de cada registro. En este caso, podríamos utilizar por ejemplo la combinación `"id"`, `"user_id"`; cuando estos campos coincidan entre la tabla fuente y la tabla destino, actualizaremos los registros en el destino, y cuando no coincidan, insertaremos los registros de la fuente en el destino. Esto lo podemos hacer utilizando los métodos del objeto `DeltaTable` dentro de la librería externa `delta-spark` que tenemos instalada en nuestro entorno virtual de Python.

In [28]:
(
    dt.alias("target")
    .merge(
        wallapop.alias("source"),
        "source.id = target.id AND source.user_id = target.user_id",
    )
    .whenMatchedUpdateAll()
    .whenNotMatchedInsertAll()
    .execute()
)

24/10/07 21:07:47 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


Cabe mencionar que también podríamos haber creado la tabla directamente a partir del DataFrame de spark, mediante la siguiente instrucción:
```
wallapop.write.format("delta").mode("overwrite").saveAsTable("wallapop")
```

Ya hemos creado la tabla delta y hemos insertado los registros tabulados de la API de Wallapop en la misma. Ahora podemos consultar el catálogo mediante SQL.

Primero, utilizaremos una función auxiliar de nuestro propio paquete de Python creado (`blackops`), llamada `get_detailed_tables_info`, para obtener información detallada de todas las tablas de nuestro catálogo de datos

In [29]:
get_detailed_tables_info(spark)

24/10/07 21:07:50 WARN ObjectStore: Failed to get database global_temp, returning NoSuchObjectException


namespace,tableName,,Catalog,Comment,Created By,Created Time,Database,InputFormat,Last Access,Location,OutputFormat,Owner,Partition Provider,Provider,Serde Library,Table,Type
,empleados,,NULL,NULL,Spark,Mon Oct 07 21:07:25 CEST 2024,NULL,NULL,UNKNOWN,NULL,NULL,NULL,NULL,NULL,NULL,empleados,VIEW
default,wallapop,,spark_catalog,Tabla de productos de Wallapop,Spark 3.5.3,Mon Oct 07 21:07:46 CEST 2024,default,org.apache.hadoop.mapred.SequenceFileInputFormat,UNKNOWN,file:/home/dadiego/projects/ESIC/esic-bigdata-iv-blackops/notebooks/tema-2-etl/spark-warehouse/wa...,org.apache.hadoop.hive.ql.io.HiveSequenceFileOutputFormat,dadiego,Catalog,delta,org.apache.hadoop.hive.serde2.lazy.LazySimpleSerDe,wallapop,MANAGED


Podemos obtener información concreta de nuestra tabla recién creada, `wallapop`

In [30]:
dt.detail()

format,id,name,description,location,createdAt,lastModified,partitionColumns,clusteringColumns,numFiles,sizeInBytes,properties,minReaderVersion,minWriterVersion,tableFeatures
delta,1abd1a96-ee4d-47ec-8b25-99662ad4c0e8,spark_catalog.default.wallapop,Tabla de productos de Wallapop,file:/home/dadiego/projects/ESIC/esic-bigdata-iv-blackops/notebooks/tema-2-etl/spark-warehouse/wa...,2024-10-07 21:07:45.344,2024-10-07 21:07:50.686,[],[],1,16479,{},1,2,"[appendOnly, invariants]"


También podemos obtener una traza histórica de las veces que esta tabla se ha modificado, lo cual es enormemente útil de cara a disponer de un gobierno del dato escalable y robusto

In [31]:
dt.history()

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
1,2024-10-07 21:07:50.686,NULL,NULL,MERGE,"{predicate -> [""((id#1132 = id#1438) AND (user_id#1134 = user_id#1440))""], matchedPredicates -> [...",NULL,NULL,NULL,0,Serializable,false,"{numTargetRowsCopied -> 0, numTargetRowsDeleted -> 0, numTargetFilesAdded -> 1, numTargetBytesAdd...",NULL,Apache-Spark/3.5.3 Delta-Lake/3.2.0
0,2024-10-07 21:07:45.572,NULL,NULL,CREATE TABLE,"{partitionBy -> [], clusterBy -> [], description -> Tabla de productos de Wallapop, isManaged -> ...",NULL,NULL,NULL,NULL,Serializable,true,{},NULL,Apache-Spark/3.5.3 Delta-Lake/3.2.0


Vemos como en el histórico aparecen las dos operaciones que hemos ejecutado sobre esta tabla Delta: la operación de creación de la tabla, y la operación de merge para insertar los nuevos datos.

Podemos también ejecutar cualquier operación SQL con esta tabla del catálogo. Por ejemplo, veamos una tabla resumen de cuántos productos existen por comunidad y código postal, ordenada de mayor a menor cantidad de productos

In [32]:
spark.sql(
    "select region, postal_code, count(*) as n_products from wallapop group by region, postal_code order by n_products desc limit 15"
)

region,postal_code,n_products
Comunidad de Madrid,28912,4
Comunidad de Madrid,28070,3
Comunidad de Madrid,28911,2
Comunidad de Madrid,28032,2
Comunidad de Madrid,28821,2
Comunidad de Madrid,28040,2
Comunidad de Madrid,28045,2
Comunidad de Madrid,28970,1
Comunidad de Madrid,28921,1
Comunidad de Madrid,28980,1
